# Automated ML

In the cell below, we import all the dependencies that we will need to complete the project.

In [ ]:
from azureml.core import Workspace, Dataset
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.pipeline.steps import AutoMLStep
from azureml.core.compute import AmlCompute
from azureml.automl.runtime.onnx_convert import OnnxConverter

## Dataset

### Overview

For our capstone project, we use [the kaggle heart failure dataset](https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data). We uploaded and registered this dataset to the workspace beforehand. The dataset stems from [a publication on heart failure prediction using machine learning](https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/s12911-020-1023-5). The dataset contains medical records for 299 patients with heart failures as well as their survival as a binary variable ("DEATH_EVENT"). The goal of our AutoML - experiment is to predict survival. Hence, we are dealing with a binary classification problem at the core of the task.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'your experiment name here'
experiment=Experiment(ws, experiment_name)

# print some information about the experiment
print(f'Workspace name: {ws.name} / AZ region: {ws.location} ' \
    f'/ Subscription ID: {ws.subscription_id} / Resource group: {ws.resource_group}')

# start logging and get the compute target
run = experiment.start_logging()
cluster_name = "capstone-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f"Found existing compute target: {compute_target}")
except Exception as e:
    print(f"Creating a new compute target (error: {e}")
    compute_cnfg = AmlCompute.provisioning_configuration(
        vm_size = "Standard_DS3_V2",
        min_nodes = 0,
        max_nodes = 4,
    )
    compute_target = ComputeTarget.create(
        ws,
        cluster_name,
        compute_cnfg,
    )
    compute_target.wait_for_completion(
        show_output=True,
        min_node_count=None,
        timeout_in_minutes=60,
    )

# message if ready
print(f'compute target: {compute_target.get_status().serialize()}')

In [ ]:
ds_name = 'heart_failure_kaggle'
dataset = Dataset.get_by_name(workspace=ws, name=ds_name)

In [ ]:
# inspect the dataframe 
dataset = dataset.to_pandas_dataframe()
dataset.head()

In [ ]:
dataset.tail()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
target_column = "DEATH_EVENT"

# automl experiment settings here
automl_settings = {
    "task": "classification",
    "primary_metric": "AUC_weighted",
    "training_data": dataset,
    "label_column_name": target_column,
    "n_cross_validations": 5,
    "compute_target": compute_target,
    "enable_early_stopping": True,
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "path": "./automl-run",
    "enable_early_stopping": True, 
    "featurization": "auto",
    "debug_log": "automl_errors.log"
}

# automl config here
automl_config = AutoMLConfig(**automl_settings)

In [ ]:
# submit the experiment
automl_run = experiment.submit(automl_config)

## Run Details

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# TODO: recheck this
best_run, best_model = automl_run.get_output()
print(f"ID of best run: {best_run.id}")
print(f"Metrics results of the best run: {best_run.get_metrics()}")
print(f"Overview over the best model and its details: {best_model}")

In [ ]:
# also retrieve the the best ONNX model and save it
best_run, best_onnx_model = automl_run.get_output(return_onnx_model=True)
onnx_model_path = "./outputs/best_model.onnx"
OnnxConverter.save_onnx_model(best_onnx_model, onnx_model_path)

In [ ]:
# TODO: Save the best model
best_run = automl_run.get_best_child()
best_model_name = best_run.properties["model_name"]
best_run.download_file(best_model_name, "./outputs/best_model.")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
best_model_name = best_run.properties["model_name"]
model = best_run.register_model(
    model_name=best_model_name, 
    model_path="./outputs"
)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# optional - depending on which model works best - see hyperparameter tuning file

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
